In [118]:
fluents = {147: {'screen': 4.511042523263396},
 175: {'door': 0.01375618393307834, 'screen': 0.12073814770214907},
 191: {'door': 1.381971717221202},
 197: {'door': 4.511042523263396},
 227: {'light': 4.111633152359644},
 231: {'light': 1.1412953889718431},
 268: {'door': 0.011047802855318829},
 272: {'door': 0.011047802855318829},
 274: {'door': 2.4837952676020367}}

In [119]:
from collections import defaultdict
import itertools
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def flatten(listOfLists):
    "Flatten one level of nesting"
    return itertools.chain.from_iterable(listOfLists)

fluents_to_recombine = defaultdict(set)
for frame in fluents:
    for fluent in fluents[frame]:
        fluents_to_recombine[fluent].add(frame)
fluents_to_recombine

defaultdict(set,
            {'door': {175, 191, 197, 268, 272, 274},
             'light': {227, 231},
             'screen': {147, 175}})

In [120]:
frames_to_recombine = fluents_to_recombine['door']
frames_to_recombine

{175, 191, 197, 268, 272, 274}

In [121]:
def split_fluents_into_windows(fluents):
    splits = list()
    kFluentOverlap = 10
    cluster = list()
    prev_frame = -kFluentOverlap
    for frame in sorted(frames_to_recombine):
        if kFluentOverlap < (frame - prev_frame):
            splits.append(cluster)
            cluster = list()
        prev_frame = frame
        cluster.append(frame)
    splits.append(cluster)
    return splits[1:]
splits = split_fluents_into_windows(frames_to_recombine)

In [122]:
split_combinations = list()
for item in splits:
    split_combinations.append(list(powerset(item))[1:])
split_combinations    

[[(175,)],
 [(191,), (197,), (191, 197)],
 [(268,), (272,), (274,), (268, 272), (268, 274), (272, 274), (268, 272, 274)]]

In [123]:
all_combos = split_combinations[0]
for i in range(1,len(split_combinations)):
    all_combos = list(list(flatten(x)) for x in itertools.product(all_combos, split_combinations[i]))
list(all_combos)

[[175, 191, 268],
 [175, 191, 272],
 [175, 191, 274],
 [175, 191, 268, 272],
 [175, 191, 268, 274],
 [175, 191, 272, 274],
 [175, 191, 268, 272, 274],
 [175, 197, 268],
 [175, 197, 272],
 [175, 197, 274],
 [175, 197, 268, 272],
 [175, 197, 268, 274],
 [175, 197, 272, 274],
 [175, 197, 268, 272, 274],
 [175, 191, 197, 268],
 [175, 191, 197, 272],
 [175, 191, 197, 274],
 [175, 191, 197, 268, 272],
 [175, 191, 197, 268, 274],
 [175, 191, 197, 272, 274],
 [175, 191, 197, 268, 272, 274]]

In [124]:
frames_to_recombine = fluents_to_recombine['light']
frames_to_recombine

{227, 231}

In [125]:
splits = split_fluents_into_windows(frames_to_recombine)

In [126]:
split_combinations = list()
for item in splits:
    split_combinations.append(list(powerset(item))[1:])
split_combinations    

[[(227,), (231,), (227, 231)]]

In [127]:
all_combos = split_combinations[0]
list(all_combos)
for i in range(1,len(split_combinations)):
    all_combos = list(list(flatten(x)) for x in itertools.product(all_combos, split_combinations[i]))
list(all_combos)

[(227,), (231,), (227, 231)]

In [128]:
powersets = dict()
for fluent in fluents_to_recombine:
    frames_to_recombine = fluents_to_recombine[fluent]
    splits = split_fluents_into_windows(frames_to_recombine)
    split_combinations = list()
    for item in splits:
        split_combinations.append(list(powerset(item))[1:])
    all_combos = split_combinations[0]
    for i in range(1,len(split_combinations)):
        all_combos = list(list(flatten(x)) for x in itertools.product(all_combos, split_combinations[i]))
    powersets[fluent] = all_combos
powersets

{'door': [[175, 191, 268],
  [175, 191, 272],
  [175, 191, 274],
  [175, 191, 268, 272],
  [175, 191, 268, 274],
  [175, 191, 272, 274],
  [175, 191, 268, 272, 274],
  [175, 197, 268],
  [175, 197, 272],
  [175, 197, 274],
  [175, 197, 268, 272],
  [175, 197, 268, 274],
  [175, 197, 272, 274],
  [175, 197, 268, 272, 274],
  [175, 191, 197, 268],
  [175, 191, 197, 272],
  [175, 191, 197, 274],
  [175, 191, 197, 268, 272],
  [175, 191, 197, 268, 274],
  [175, 191, 197, 272, 274],
  [175, 191, 197, 268, 272, 274]],
 'light': [(227,), (231,), (227, 231)],
 'screen': [[147, 175]]}

In [129]:
powerset_counts = map(lambda x: (x,len(powersets[x]),),powersets.keys())
powerset_counts

[('light', 3), ('screen', 1), ('door', 21)]

In [218]:
powerset_count_split_combinations = map(lambda x: range(0,x[1]),powerset_counts)
powerset_count_split_combinations

[[0, 1, 2],
 [0],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]

In [245]:
if len(powerset_count_split_combinations) > 1:
    merge_combinations = list(itertools.product(powerset_count_split_combinations[0], powerset_count_split_combinations[1]))
    for item in powerset_count_split_combinations[2:]:
        merge_combinations = list(itertools.product(merge_combinations, item))
        merge_combinations = list(list(flatten(x)) for x in map(lambda x: (x[0],(x[1],)),merge_combinations))
    list(merge_combinations)
else:
    merge_combinations = powerset_count_split_combinations[0]
combination = merge_combinations[1]
combination
zip(powerset_counts,combination)

[(('light', 3), 0), (('screen', 1), 0), (('door', 21), 1)]

In [251]:
parses_to_recombine = dict(map(lambda x:(x[0][0],powersets[x[0][0]][x[1]]),zip(powerset_counts,combination)))
parses_to_recombine

{'door': [175, 191, 272], 'light': (227,), 'screen': [147, 175]}

In [253]:
recombined_parses = defaultdict(dict)
for fluent in parses_to_recombine:
    for frame in parses_to_recombine[fluent]:
        recombined_parses[frame][fluent] = fluents[frame][fluent]
recombined_parses

defaultdict(dict,
            {147: {'screen': 4.511042523263396},
             175: {'door': 0.01375618393307834, 'screen': 0.12073814770214907},
             191: {'door': 1.381971717221202},
             227: {'light': 4.111633152359644},
             272: {'door': 0.011047802855318829}})

In [4]:
powerset_counts = [('screen', 1)]
powersets = {'screen': [(209,)]}
zip(powerset_counts,(0,))


[(('screen', 1), 0)]

In [7]:
combination = 0
print type(0)
print type(combination)

<type 'int'>
<type 'int'>
